<a href="https://colab.research.google.com/github/vaguiar/CodingSamples/blob/master/hw2/hw2_collab_filtering_exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display, Markdown, HTML
import os

In [2]:
display(Markdown("### Mounting GDrive in Colab" ))
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

#### Mounting GDrive in Colab

Mounted at /content/drive


In [4]:
def get_data(file):
  data = pd.read_csv(f"/content/drive/My Drive/Personalization/hw2/data/ml-latest/{file}")
  display(Markdown(f"{file} has %s columns & %s rows" % ( len(data.columns), len(data.index) ) ))
  return data

display(Markdown("### Reading data"))
movies = get_data("movies.csv")
ratings = get_data("ratings.csv")

#### Reading data

movies.csv has 3 columns & 58098 rows

ratings.csv has 4 columns & 27753444 rows

In [5]:
def get_popularity(ratings_df, id_col):
    # count the number of ratings 
    popularity_df = ratings_df[id_col].value_counts().to_frame().reset_index()
    popularity_df.columns = [id_col, 'count']

    # assign weights
    popularity_df['weights'] = 1/(1-popularity_df['count'].map(popularity_df['count'].value_counts(normalize = True)))

    return popularity_df

def sampling_id(popularity_df, id_col, threshold, num, random_state=123):
    # filter by threshold
    valid_df = popularity_df[popularity_df['count'] >= threshold] 
    # sample by weights 
    sampled_id = valid_df.sample(n = num, weights = 'weights', random_state=random_state)[id_col].tolist()
    
    return sampled_id

In [6]:
# initiate constants
MOVIE_THRESHOLD = 50
USER_THRESHOLD = 50

# get movie rating counts df & sampled movie_ids
popularity_movie_df = get_popularity(ratings, id_col='movieId')
sampled_movieId = sampling_id(popularity_movie_df, id_col='movieId', threshold=MOVIE_THRESHOLD, num=1500)

# get user rating counts df & sampled user_ids
popularity_user_df = get_popularity(ratings, id_col='userId')
sampled_userId = sampling_id(popularity_user_df, id_col='userId', threshold=USER_THRESHOLD, num=25000)

In [9]:
display(Markdown("#### Sampled data"))

# select by sampled movieId
sampled_df = ratings[ratings['movieId'].isin(sampled_movieId)]
# select by sampled userId
sampled_df = sampled_df[sampled_df['userId'].isin(sampled_userId)]

#### Sampled data

In [10]:
sampled_df

,userId,movieId,rating,timestamp
2122,31,608,5.0,968886879
2135,31,1240,4.0,968887413
2144,31,1304,3.0,968870967
2154,31,2396,5.0,968886749
2158,31,2716,4.0,968871098
...,...,...,...,...
27752635,283215,276,4.0,862414494
27752642,283215,494,5.0,854982148
27752648,283215,608,3.0,854982147
27752664,283215,788,5.0,854982198


### Surprise - KNN User based recommendations

In [11]:
# Only once per session
!pip install scikit-surprise

     |████████████████████████████████| 11.8MB 5.5MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1670927 sha256=584909681cae976de3a9f8b3c8a540442e8ad69f7217f42f4de3f06a640ea73b
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [12]:
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split, cross_validate
from surprise import KNNWithMeans
from surprise import accuracy

# Step 1 = Data Import & Prep
# ratings_full = get_data("ratings.csv")
ratings_full = sampled_df
ratings = ratings_full.iloc[:20000]

reader = Reader(rating_scale=(0.5,5))

data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [13]:
# Create the train test split
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.2, shuffle=True)

In [14]:
## If we had the use data as a training chunk
# # trainsetfull = data.build_full_trainset()

display(Markdown("## Training Set:"))
print('Number of ratings: ', trainset.n_ratings)
print('Number of users: ', trainset.n_users)
print('Number of items: ', trainset.n_items)
print('Global Mean:', trainset.global_mean )

display(Markdown("## Test Set:"))
print('Number of ratings: ', len(testset), '\n')

## Training Set:

Number of ratings:  16000
Number of users:  751
Number of items:  1139
Global Mean: 3.576375


## Test Set:

Number of ratings:  4000 



In [17]:
# Step 2 - Cross-Validation

my_k = 25
my_min_k = 5    # if min_k is not satisfied the global average will be used for estimates. (default 1)
my_sim_option = {
    'name':'pearson', 'user_based':True
    }


algo = KNNWithMeans(
  k = my_k, 
  min_k = my_min_k, 
  sim_options = my_sim_option, 
  verbose = True
  )
  
results = cross_validate(
    algo = algo, 
    data = data, 
    measures=['RMSE', 'MAE', 'MSE'], 
    cv=5, 
    return_train_measures=True
    )
    
print("RMSE: ", results['test_rmse'].mean())
print("MAE: ", results['test_mae'].mean())
print("MSE: ", results['test_mse'].mean())

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE:  0.9236086113338541
MAE:  0.7064122634957551
MSE:  0.8531533330892632


In [71]:
def algo_name(algo):
  return str(algo).split(".")[-1].split("'")[0]

def try_algo(algo_fn, k_val, min_k_val=5, similarity_method='pearson', user_based=True):

  my_k = k_val
  my_min_k = min_k_val
  my_sim_option = {
      'name':similarity_method, 
      'user_based':user_based
      }

  algo = algo_fn(
    k = my_k, 
    min_k = my_min_k, 
    sim_options = my_sim_option, 
    verbose = True
    )
  
  display(Markdown("#### Running '%s' algo (user based=%s) '%s' similarity" %(algo_name(algo_fn), user_based, similarity_method)))

  results = cross_validate(
      algo = algo, 
      data = data, 
      measures=['RMSE', 'MAE', 'MSE'], 
      cv=5, 
      return_train_measures=True
      )

  display(Markdown("#### Results:"))
  print("RMSE: ", results['test_rmse'].mean(), "MAE: ", results['test_mae'].mean(), "MSE: ", results['test_mse'].mean())

  return results

display(Markdown("### A single movie in the sampled dataset has atleast 30 user ratings. Hence while finding computing the KNN similarities, with can use a k with value upto 30. I will use 25"))

### A single movie in the sampled dataset has atleast 30 user ratings. Hence while finding computing the KNN similarities, with can use a k with value upto 30. I will use 25

In [72]:
from surprise import KNNWithMeans, KNNWithZScore, KNNBaseline, KNNBasic

algos = [ KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline ]
sims = ['cosine', 'msd', 'pearson', 'pearson_baseline']
user_based_opts = [ True, False ]
k_val = 25

display(Markdown("## Cross Validating to find best Memory Based option"))
display(Markdown("k-NN based nearest neighbor algorithm with k value of '%s'" %(str(k_val))))

userBasedOp = pd.DataFrame(index=[algo_name(i) for i in algos], columns=sims)
itemBasedOp = pd.DataFrame(index=[algo_name(i) for i in algos], columns=sims)

for algo in algos:
  for sim in sims:
    for user_based_opt in user_based_opts:

      rslt = try_algo(algo, k_val=k_val, similarity_method=sim, user_based=user_based_opt)
      
      if user_based_opt:
        userBasedOp.at[algo_name(algo), sim] = rslt['test_rmse'].mean()
      else:
        itemBasedOp.at[algo_name(algo), sim] = rslt['test_rmse'].mean()

## Cross Validating to find best Memory Based option

k-NN based nearest neighbor algorithm with k value of '25'

#### Running 'KNNBasic' algo (user based=True) 'cosine' similarity

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


#### Results:

...

RMSE:  0.8932172950050787 MAE:  0.6865872124600345 MSE:  0.7980038160250855


#### Running 'KNNBaseline' algo (user based=False) 'pearson_baseline' similarity

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


#### Results:

RMSE:  0.8984952447060446 MAE:  0.6878728580988007 MSE:  0.8073658718013734


In [80]:
userBasedOp

,cosine,msd,pearson,pearson_baseline
KNNBasic,0.965819,0.942553,0.972989,0.960737
KNNWithMeans,0.919603,0.916919,0.929083,0.923177
KNNWithZScore,0.921973,0.921003,0.925156,0.92566
KNNBaseline,0.894374,0.891783,0.901363,0.893217


In [73]:
itemBasedOp

,cosine,msd,pearson,pearson_baseline
KNNBasic,0.972441,0.950242,0.982154,0.970407
KNNWithMeans,0.910834,0.905003,0.924228,0.925072
KNNWithZScore,0.91529,0.912774,0.925798,0.923268
KNNBaseline,0.900281,0.893157,0.901617,0.898495


In [ ]:
# Train model and predicts with train/test data cuts separately.

algo = KNNBaseline(
    k=15,
    min_k=5,
    sim_options={
      'name':'pearson', 'user_based':True
    },
    verbose=True
)

algo.fit(trainset)
predictions = algo.test(testset)

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.


In [ ]:
accuracy.rmse(predictions)

RMSE: 0.9391


0.9391173955433683

In [ ]:
print(testset)

[(200, 79132, 4.0), (43, 497, 4.5), (138, 1089, 5.0), (4, 2011, 4.0), (73, 48877, 0.5), (38, 3082, 4.0), (158, 367, 3.0), (138, 1649, 5.0), (183, 589, 4.0), (73, 48082, 4.5), (35, 1214, 4.0), (19, 482, 3.0), (56, 1586, 4.0), (101, 150, 3.0), (163, 364, 3.0), (176, 5608, 4.0), (93, 1233, 4.5), (56, 6373, 3.0), (11, 2302, 3.5), (55, 1082, 3.5), (181, 356, 5.0), (174, 4896, 3.5), (119, 194, 5.0), (189, 2268, 3.0), (55, 2140, 4.5), (180, 2788, 5.0), (81, 2628, 3.0), (98, 2428, 3.5), (100, 3536, 3.5), (88, 8783, 4.0), (32, 112552, 4.5), (95, 1222, 4.0), (16, 224, 4.0), (158, 761, 3.0), (134, 3088, 4.0), (51, 1259, 4.0), (138, 6535, 2.0), (173, 3784, 3.0), (36, 4776, 3.5), (59, 161, 4.0), (180, 356, 4.0), (100, 919, 4.5), (99, 101739, 3.0), (48, 68237, 5.0), (51, 1321, 4.0), (51, 2421, 3.0), (81, 58559, 4.0), (163, 1397, 4.0), (93, 1653, 2.0), (164, 377, 4.0), (67, 1544, 2.0), (134, 105468, 4.0), (114, 164, 3.0), (155, 3513, 4.0), (138, 2657, 0.5), (147, 45720, 2.0), (183, 919, 5.0), (173, 3

In [ ]:

def test_user_mov(user, movie):
  print(algo.predict(uid = user, iid = movie))
  print("User seen in training:", trainset.knows_user(user))
  print("Movie seen in training:", trainset.knows_user(movie))

In [ ]:
test_user_mov(200, 79132)

user: 200        item: 79132      r_ui = None   est = 4.23   {'actual_k': 7, 'was_impossible': False}
User seen in training: True
Movie seen in training: False


In [ ]:
algo.predict(uid = 73, iid = 48877)

Prediction(uid=73, iid=48877, r_ui=None, est=3.274914446592719, details={'actual_k': 3, 'was_impossible': False})

In [ ]:
trainset.knows_user(73)

True

In [ ]:
trainset.knows_item(48877)

False

In [ ]:
algo.predict(uid = 38, iid = 3082)

Prediction(uid=38, iid=3082, r_ui=None, est=3.845218885423006, details={'actual_k': 6, 'was_impossible': False})

In [ ]:
ratings_full.iloc[20001:20010]

,userId,movieId,rating,timestamp
20001,203,7460,5.0,1330146582
20002,203,8011,5.0,1330146810
20003,203,8645,4.5,1330147976
20004,203,8910,4.5,1330149390
20005,203,8949,4.0,1330149392
20006,203,30810,5.0,1330147178
20007,203,41571,4.0,1330145544
20008,203,44195,4.5,1330146885
20009,203,48043,4.0,1330145449


In [ ]:
trainsetfull.knows_user(1)

True

In [ ]:
# Step 3 - Model Fitting
algo.fit(trainset)

TypeError: ignored

In [ ]:
algo.predict(uid = 203, iid = 8645)

Prediction(uid=203, iid=8645, r_ui=None, est=3.25, details={'actual_k': 1, 'was_impossible': False})